In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
if len(gpu) > 0:
    tf.config.experimental.set_memory_growth(gpu, True)
from src.feature.data_loader import DataLoader
import json
import os

In [2]:
import mlflow
import mlflow.tensorflow
from mlflow.tracking import MlflowClient

In [3]:
BATCH_SIZE = 32

data_loader = DataLoader(shape=(360,360), batch_size = BATCH_SIZE) 
data_loader.load_from_directory("/home/mugesh/IB/cls/processed_data")
data_gen = data_loader.data_generator()

Loaded 713 files of 4 classes


In [4]:
## Model Architecture
input_ = tf.keras.layers.Input(shape=(360,360,1), name ="input")

conv_1 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = "same", name="conv_1")(input_)
act_1 = tf.keras.layers.Activation('relu', name='act_1')(conv_1)
pool_1 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_1")(act_1)

conv_2 = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding = "same" , name="conv_2")(pool_1)
act_2 = tf.keras.layers.Activation('relu', name='act_2')(conv_2)
pool_2 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_2")(act_2)

conv_3 = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding = "same",  name="conv_3")(pool_2)
act_3 = tf.keras.layers.Activation('relu', name='act_3')(conv_3)
pool_3 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_3")(act_3)

conv_4 = tf.keras.layers.Conv2D(filters = 16, kernel_size = (3,3), padding = "same", name="conv_4")(pool_3)
act_4 = tf.keras.layers.Activation('relu', name='act_4')(conv_4)
pool_4 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_4")(act_4)
pool_5 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_5")(pool_4)

flatten = tf.keras.layers.Flatten(name="flatten")(pool_5)


dense_1 = tf.keras.layers.Dense(64, activation='relu', name = "dense_1")(flatten)
dense_2 = tf.keras.layers.Dense(32, activation='relu', name = "dense_2")(dense_1)

shaded = tf.keras.layers.Dense(1,activation='sigmoid', name="shaded")(dense_2)
gridline = tf.keras.layers.Dense(1,activation='sigmoid', name="gridline")(dense_2)
good = tf.keras.layers.Dense(1,activation='sigmoid', name="good")(dense_2)
black_border = tf.keras.layers.Dense(1,activation='sigmoid', name="black_border")(dense_2)

model_outputs = {'black_border' : black_border, 'good' : good, 'gridline' : gridline, 'shaded' : shaded}


model = tf.keras.models.Model(input_, model_outputs, name ="DockNet")
model.summary()

## Compile 
model.compile(
    loss={
        "shaded": 'binary_crossentropy',
        "gridline": "binary_crossentropy",
        "good": "binary_crossentropy",
        "black_border": "binary_crossentropy"
    },
    optimizer='adam',
    metrics=['accuracy']
)

Model: "DockNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 360, 360, 1) 0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 360, 360, 64) 640         input[0][0]                      
__________________________________________________________________________________________________
act_1 (Activation)              (None, 360, 360, 64) 0           conv_1[0][0]                     
__________________________________________________________________________________________________
pool_1 (MaxPooling2D)           (None, 180, 180, 64) 0           act_1[0][0]                      
____________________________________________________________________________________________

In [5]:
experiment_name = "DockNet"

In [6]:
mlflow.tensorflow.autolog()
with mlflow.start_run(run_name=experiment_name):
    history = model.fit(
        data_gen,
        steps_per_epoch= data_loader.num_files // BATCH_SIZE,
        batch_size = BATCH_SIZE,
        epochs=10)

Epoch 1/10
22/22 [==============================] - 5s 44ms/step - loss: 2.4531 - black_border_loss: 0.5936 - good_loss: 0.6619 - gridline_loss: 0.5107 - shaded_loss: 0.6869 - black_border_accuracy: 0.7489 - good_accuracy: 0.6946 - gridline_accuracy: 0.7722 - shaded_accuracy: 0.6219
Epoch 2/10
22/22 [==============================] - 1s 41ms/step - loss: 1.8371 - black_border_loss: 0.4707 - good_loss: 0.3561 - gridline_loss: 0.5237 - shaded_loss: 0.4867 - black_border_accuracy: 0.7522 - good_accuracy: 0.9232 - gridline_accuracy: 0.8394 - shaded_accuracy: 0.7591
Epoch 3/10
22/22 [==============================] - 1s 44ms/step - loss: 0.5635 - black_border_loss: 0.2112 - good_loss: 0.0702 - gridline_loss: 0.2481 - shaded_loss: 0.0341 - black_border_accuracy: 0.8760 - good_accuracy: 1.0000 - gridline_accuracy: 0.8984 - shaded_accuracy: 1.0000
Epoch 4/10
22/22 [==============================] - 1s 41ms/step - loss: 0.0766 - black_border_loss: 0.0161 - good_loss: 0.0016 - gridline_loss: 0.0

2022/03/04 16:38:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpoemu10bo/model, flavor: keras), fall back to return ['tensorflow==2.4.2']. Set logging level to DEBUG to see the full traceback.
